In [82]:
import doctest
from dataclasses import dataclass
import numpy as np
import re
import math

In [283]:
_int_re = re.compile(r'\d+')
def extract_ints(s: str) -> list[int]:
  """
  >>> extract_ints("Button A: X+94, Y+34")
  [94, 34]
  >>> extract_ints("Prize: X=8400, Y=5400")
  [8400, 5400]
  """
  return [int(m) for m in _int_re.findall(s)]


@dataclass(frozen=True)
class ButtonConfiguration:
  vectors: np.array
  target: np.array
  
  @classmethod
  def parse_input(cls, input: str) -> 'ButtonConfiguration':
    button_a, button_b, prize = input.strip().splitlines()
    vectors = np.array([
      extract_ints(button_a), 
      extract_ints(button_b)
    ])
    target = np.array(extract_ints(prize))
    return ButtonConfiguration(vectors, target)


@dataclass
class ProblemInput:
  configs: list[ButtonConfiguration]
  
  @classmethod
  def parse_input(cls, input: str) -> 'ProblemInput':
    return ProblemInput(
      [ButtonConfiguration.parse_input(s) for s in input.split('\n\n')]
    )


def is_valid_solution(bc: ButtonConfiguration, presses: np.array, offset: int=0) -> bool:
  int_presses = presses.round().astype(int)
  deltas = np.abs(int_presses.dot(bc.vectors) - (bc.target + offset))
  return all(np.isclose(deltas, 0, atol=1e-4) & (presses >= 0))


def token_cost(presses: np.array) -> int:
  """
  >>> token_cost(np.array([80., 40.0]))
  280
  """
  return int(presses.round().dot([3, 1]))
  

def minimum_tokens(bc: ButtonConfiguration, offset: int=0) -> int:
  presses = (bc.target + offset).dot(np.linalg.inv(bc.vectors))
  if is_valid_solution(bc, presses, offset):
    return token_cost(presses)
  else:
    return 0

In [284]:
def part_1_solution(p: ProblemInput) -> int:
  return sum(minimum_tokens(bc) for bc in p.configs)


def part_2_solution(p: ProblemInput) -> int:
  return sum(minimum_tokens(bc, 10000000000000) for bc in p.configs)
  

In [285]:
doctest.testmod(verbose=False, report=True, exclude_empty=True, optionflags=doctest.NORMALIZE_WHITESPACE)

TestResults(failed=0, attempted=3)

In [286]:
test_input = """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""

problem = ProblemInput.parse_input(test_input)
assert part_1_solution(problem) == 480, "p1 test failed"

In [287]:
part_2_solution(problem)

875318608908

In [290]:
%%time
# Final answers
with open('inputs/day13.txt') as f:
    input = f.read().strip()
    problem = ProblemInput.parse_input(input)
    print('Part 1: ', part_1_solution(problem))
    print('Part 2: ', part_2_solution(problem))

Part 1:  28753
Part 2:  102718967795500
CPU times: user 35.2 ms, sys: 0 ns, total: 35.2 ms
Wall time: 33.6 ms
